#### Pacotes importados para manipulação dos dados

In [1]:
import pandas as pd

##### Importação dos dados

In [4]:
sample_submission = pd.read_csv(r'dados\taxi-sample-submission.csv')
submission = pd.read_csv(r'dados\taxi-submission.csv')
taxi_train = pd.read_csv(r'dados\taxi-train.csv')

In [14]:
taxi_train.head(10)

,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location,ride_date
0,5c41b7afe9da0d7fbe60dc0e925975b832375a3c,efd204f38a83a59df5bf4a1fe004aa291183e235ee40ee...,2016-02-07 01:45:00+00:00,2016-02-07 01:45:00+00:00,360.0,0.00,1.703108e+10,1.703108e+10,8.0,8.0,...,8.50,Credit Card,Northwest Management LLC,41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),41.900221,-87.629105,POINT (-87.6291051864 41.9002212967),2016-02-07
1,9333962b69bef1506be0b0a7edbd1d6626edd6ab,1a1189d1e9f4f676bbeaf49903912684b94453576a234f...,2013-05-08 19:15:00+00:00,2013-05-08 19:30:00+00:00,840.0,5.40,NaN,NaN,NaN,NaN,...,13.85,Cash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-05-08
2,6329807d0ad342bfe6671b2553592edadfd742b6,25e9bbb9704eb82311a4d4036baba6451cd27138c4d6eb...,2013-02-17 12:30:00+00:00,2013-02-17 12:30:00+00:00,NaN,0.00,NaN,NaN,NaN,NaN,...,52.38,Credit Card,Chicago Elite Cab Corp.,NaN,NaN,NaN,NaN,NaN,NaN,2013-02-17
3,049949c0f58cc75d58ef9faf8ba4030c6ef0cf34,8b8b185d8a60ac5edd94c0494d0b9b9c9c86d54c0ab3dd...,2014-11-09 10:45:00+00:00,2014-11-09 11:00:00+00:00,540.0,1.60,NaN,NaN,6.0,3.0,...,9.55,Credit Card,NaN,41.944227,-87.655998,POINT (-87.6559981815 41.9442266014),41.965812,-87.655879,POINT (-87.6558787862 41.96581197),2014-11-09
4,4554e53540678f18384d5fcd6d42623ccc60d6e2,73b2f5adecea91eeef3900303a07f1b0519a594cffb6b0...,2019-04-13 20:00:00+00:00,2019-04-13 20:00:00+00:00,577.0,1.50,1.703184e+10,1.703108e+10,32.0,8.0,...,11.25,Credit Card,Chicago Taxicab,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),2019-04-13
5,0c9fd40664d8961476c8f96747498a62488e59d2,b9ce4a1df5c8bf18e20212f8310e1b1e72b44dff6c9157...,2014-09-28 10:45:00+00:00,2014-09-28 10:45:00+00:00,360.0,0.80,1.703108e+10,1.703108e+10,8.0,8.0,...,9.65,Credit Card,NaN,41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),2014-09-28
6,d0e6dfdaa8ab539955678f75822c32d5a9e8e496,e88bf49a9bb5b04a354628076eed4fcffc39fc2632a5bb...,2017-05-27 10:30:00+00:00,2017-05-27 10:45:00+00:00,535.0,2.29,1.703184e+10,1.703132e+10,33.0,32.0,...,12.75,Credit Card,Chicago Carriage Cab Corp,41.849247,-87.624135,POINT (-87.6241352979 41.84924675450001),41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),2017-05-27
7,4620c514566fea08bfdfb83a3fa5559ce086106c,88a493149cafbde5ddde0280c77cc90c26a015c260df78...,2015-03-26 09:00:00+00:00,2015-03-26 09:15:00+00:00,950.0,1.60,1.703128e+10,1.703108e+10,28.0,8.0,...,10.25,Credit Card,American United,41.885300,-87.642808,POINT (-87.6428084655 41.8853000224),41.890922,-87.618868,POINT (-87.6188683546 41.8909220259),2015-03-26
8,974e5b43133104d182bcc467c0a000415537bf8d,f3e4a7887f56443a5f50669bac8c3db338a77b797facaa...,2018-12-03 06:45:00+00:00,2018-12-03 06:45:00+00:00,360.0,1.10,1.703184e+10,1.703108e+10,32.0,8.0,...,11.25,Credit Card,Medallion Leasin,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.895033,-87.619711,POINT (-87.6197106717 41.8950334495),2018-12-03
9,f190d8c105ac5a844f2904dffb6e0d9604299557,2845f8fe2353cdc20ae1956d27c69e4618a96ad543b853...,2014-06-13 16:30:00+00:00,2014-06-13 16:30:00+00:00,480.0,1.20,1.703184e+10,1.703183e+10,32.0,28.0,...,9.05,Credit Card,NaN,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.879067,-87.657005,POINT (-87.657005027 41.8790669938),2014-06-13


In [21]:
taxi_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   unique_key              30000 non-null  object 
 1   taxi_id                 29998 non-null  object 
 2   trip_start_timestamp    30000 non-null  object 
 3   trip_end_timestamp      29996 non-null  object 
 4   trip_seconds            29805 non-null  float64
 5   trip_miles              30000 non-null  float64
 6   pickup_census_tract     19663 non-null  float64
 7   dropoff_census_tract    19559 non-null  float64
 8   pickup_community_area   26445 non-null  float64
 9   dropoff_community_area  25919 non-null  float64
 10  fare                    29999 non-null  float64
 11  tips                    29999 non-null  float64
 12  tolls                   24933 non-null  float64
 13  extras                  29999 non-null  float64
 14  trip_total              29999 non-null

In [20]:
taxi_train.groupby('company')[['trip_total']].sum().sort_values(by='trip_total', ascending=False)

,trip_total
company,
Taxi Affiliation Services,90309.07
Flash Cab,41215.12
Chicago Carriage Cab Corp,23628.55
Yellow Cab,22938.13
Dispatch Taxi Affiliation,22453.37
...,...
3591- Chuk's Cab,6.85
Peace Taxi Assoc,6.00
5724 - 72965 KYVI Cab Inc,5.65
